In [ ]:
!pip install spacy

In [ ]:
pip install pytextrank

In [ ]:
!python -m spacy download en_core_web_sm

In [4]:
import spacy
from spacy.matcher import Matcher
from collections import Counter

from icecream import ic

import pytextrank

import re

import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel

import numpy as np

import pyLDAvis
import pyLDAvis.gensim_models

In [5]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [6]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
nlp.add_pipe("textrank", config={ "stopwords": { "word": ["NOUN"] } })

In [8]:
# Open the file
with open("PsycOfTheUncon.txt", "r", encoding="utf8") as f:
    text = f.read()

In [9]:
# Clean the text
text = text.replace('\n', ' ')
text = text.replace('_',' ')
text = re.sub("\[\d+\]", '', text)

In [10]:
# add some additional stop words
my_stop_words = ['mr', 'Mr', 'Miss', 'great', 'like']

for stopword in my_stop_words:
    lexeme = nlp.vocab[stopword]
    lexeme.is_stop = True

In [11]:
doc = nlp(text)

# Look at entities in the text

In [ ]:
# arbitrarily looked into entities in the text in hope of finding anything interesting
for ent in doc.ents:
    if ent.label_ != 'DATE' and ent.label_ != 'CARDINAL' and ent.label_ != 'TIME':
        print(ent.text, ent.label_)

In [ ]:
# noticed many 'Christian's, therefore decided to go deeper into a label 'NORP'
for ent in doc.ents:
    if ent.label_ == 'NORP':
        print(ent.text, ent.label_)

In [14]:
# fetch the 10 most common 'NORP' words
norps = [token.text
         for token in doc.ents
         if (token.label_ == 'NORP')]
norp_freq = Counter(norps)

common_norps = norp_freq.most_common(10)
common_norps

[('Christian', 78),
 ('Egyptian', 32),
 ('Christianity', 23),
 ('Indian', 19),
 ('Latin', 19),
 ('German', 18),
 ('Greek', 17),
 ('Freudian', 11),
 ('Roman', 9),
 ('Eleusinian', 9)]

In [15]:
# mini-conclusion: the text probably has a lot to do with Christianity and ancient civilizations, as may conclude from words such as 
#'Egyptian', 'Indian', 'Greek', 'Roman', 'Eleusinian'

# Try Summarising

In [16]:
# try summarising the text using PyTextRank 

# seperate the text by chapter(13 in total, introduction included)
texts, article = [], ""

for word in doc:
    article = article + " " + word.text
    if word.text == 'CHAPTER':
        texts.append(article)
        article = ""
texts.append(article)

In [17]:
# look at top-ranked phrases in each chapter
doclist = []

for subdoc in range(len(texts)):
    dl = nlp(texts[subdoc])
    doclist.append(dl)
    
for subdoc in doclist: 
    for p in subdoc._.phrases[:10]:
        print('{:.4f} {:5d}  {}'.format(p.rank, p.count, p.text))
        print(p.chunks)
    print("--------------------------------")

0.0743     1  individual psychologic problems
[individual psychologic problems]
0.0632     1  historical material
[historical material]
0.0631     1  historical problems
[historical problems]
0.0627     1  individual psychology
[individual psychology]
0.0616     1  specific historical material
[specific historical material]
0.0512     5  Freud
[Freud, Freud, Freud, Freud, Freud]
0.0508     1  Oedipus
[Oedipus]
0.0504     1  unconscious work
[unconscious work]
0.0471     1  new light
[new light]
0.0460     1  psychoanalytic knowledge
[psychoanalytic knowledge]
--------------------------------
0.0578     9  directed thinking
[directed thinking, directed thinking, directed thinking, directed thinking, directed thinking, directed thinking, directed thinking, directed thinking, directed thinking]
0.0559     1  dream psychology
[dream psychology]
0.0550     1  Dreams
[Dreams]
0.0550    13  dreams
[dreams, dreams, dreams, dreams, dreams, dreams, dreams, dreams, dreams, dreams, dreams, dreams,

In [18]:
# summarise each chapter by yielding each top 3 sentences
for subdoc in doclist:
    tr = subdoc._.textrank
    for sent in tr.summary(limit_phrases=15, limit_sentences=3):
        ic(sent)
    ic("--------------------------------")

ic| sent: The leading purpose of these works is the unlocking of historical problems through the application of psychoanalytic knowledge ; that is to say , knowledge drawn from the activity of the modern unconscious mind concerning specific historical material .
ic| sent: For , just as the psychoanalytic conceptions promote understanding of the historic psychologic creations , so reversedly historical materials can shed new light upon individual psychologic problems .
ic| sent: It seems to me , however , that in the present state of affairs there is a more or less imperative demand for the psychoanalyst to broaden the analysis of the individual problems by a comparative study of historical material relating to them , just as Freud has already done in a masterly manner in his book on “ Leonardo da Vinci . ”
ic| '--------------------------------'
ic| sent: While directed thinking is a phenomenon conscious throughout , the same can not be asserted of phantastic thinking .
ic| sent: By mea

In [19]:
# mini-conclusion: as summarised in the top 3 sentences of the introduction part of the book(the first 3 sentences), the overall text 
# seems to focus on examining the relation between historical materials and the psychology of an individual, and probably how they have been 
# influenced by each other since the ancient time, which may explains the top 'NORP' words from the previous section. The text seems to discuss 
# in details about religions and ancient myths, as well as their symbolic meanings, especially with regards to libido and Mother. 

# Try Topic Modeling

In [20]:
# try extracting topics using gensim and LDA model

# seperate the text by chapter
texts, article = [], []

for word in doc:
    if word.pos_ != 'SPACE' and word.pos_ != 'VERB' and not word.is_stop and not word.is_punct and not word.like_num:
        article.append(word.lemma_)
        
    if word.text == 'CHAPTER':
        texts.append(article)
        article = []
texts.append(article)

In [21]:
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [22]:
lda_model = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary)
lda_model.show_topics()

[(0,
  '0.010*"mother" + 0.008*"libido" + 0.004*"world" + 0.004*"man" + 0.004*"sun" + 0.004*"symbol" + 0.004*"death" + 0.004*"god" + 0.004*"life" + 0.004*"time"'),
 (1,
  '0.017*"mother" + 0.010*"libido" + 0.006*"life" + 0.005*"symbol" + 0.005*"sun" + 0.005*"world" + 0.005*"man" + 0.004*"death" + 0.004*"God" + 0.004*"god"'),
 (2,
  '0.008*"mother" + 0.007*"libido" + 0.005*"world" + 0.005*"symbol" + 0.004*"man" + 0.004*"sun" + 0.004*"God" + 0.004*"time" + 0.004*"fire" + 0.004*"life"'),
 (3,
  '0.010*"libido" + 0.010*"mother" + 0.006*"sun" + 0.005*"man" + 0.005*"life" + 0.004*"God" + 0.004*"hero" + 0.004*"world" + 0.004*"sexual" + 0.003*"time"'),
 (4,
  '0.013*"mother" + 0.006*"libido" + 0.005*"man" + 0.005*"world" + 0.004*"life" + 0.004*"time" + 0.004*"death" + 0.004*"tree" + 0.004*"God" + 0.004*"hero"'),
 (5,
  '0.011*"libido" + 0.011*"mother" + 0.006*"man" + 0.006*"world" + 0.005*"life" + 0.004*"time" + 0.004*"hero" + 0.004*"God" + 0.004*"symbol" + 0.004*"god"'),
 (6,
  '0.009*"mother

In [23]:
# visualize the result using pyLDAvis
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.001751  0.004615       1        1  48.881252
5     -0.003925  0.003449       2        1  29.484540
1      0.008371  0.002205       3        1  16.092157
4      0.001602 -0.003841       4        1   2.659568
7      0.002572 -0.003507       5        1   1.462307
3     -0.003947  0.003972       6        1   1.408514
2     -0.002555 -0.001952       7        1   0.003076
6     -0.001789 -0.000492       8        1   0.002862
9     -0.003479 -0.003732       9        1   0.002862
0      0.001398 -0.000717      10        1   0.002862, topic_info=           Term        Freq       Total Category  logprob  loglift
199      mother  558.000000  558.000000  Default  30.0000  30.0000
996      libido  400.000000  400.000000  Default  29.0000  29.0000
186         man  302.000000  302.000000  Default  28.0000  28.0000
1380        sun  234.000000  234.000000  Default  27.0000  27.0000
1391     symbol  228.000000  228.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
2551  sacrifice    0.002710   98.303149  Topic10  -6.1786  -0.0373
1501      water    0.002729  104.910724  Topic10  -6.1718  -0.0955
1447       tree    0.002868  125.891436  Topic10  -6.1222  -0.2282
1027    meaning    0.002773  145.532253  Topic10  -6.1558  -0.4069
379         God    0.002805  170.629347  Topic10  -6.1443  -0.5544

[877 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         1  0.352253   Abraham
0         2  0.352253   Abraham
0         3  0.234835   Abraham
5686      1  0.465260      Adam
5686      2  0.186104      Adam
...     ...       ...       ...
6836      1  0.765644  σύμβολον
7361      1  0.504979        ἐς
7361      3  0.504979        ἐς
3710      1  0.713662     ἡλίου
3710      2  0.356831     ἡλίου

[1538 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 6, 2, 5, 8, 4, 3, 7, 10, 1])

In [24]:
# Try Bigram
bigram = gensim.models.Phrases(texts, min_count=5, threshold=100)
textsbi = [bigram[line] for line in texts]

In [25]:
dictionarybi = Dictionary(textsbi)
corpusbi = [dictionarybi.doc2bow(text) for text in textsbi]

In [ ]:
lda_model_bi = LdaModel(corpus=corpusbi, num_topics=10, id2word=dictionarybi)
lda_model_bi.show_topics()

In [27]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(lda_model_bi, corpusbi, dictionarybi)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.008218  0.001128       1        1  39.547769
6      0.008172 -0.001747       2        1  17.736334
9      0.000160 -0.002521       3        1  12.588034
5     -0.005491 -0.006806       4        1   9.858129
0     -0.000169  0.000301       5        1   7.086978
8     -0.008953  0.002142       6        1   6.946695
4      0.002166 -0.000442       7        1   3.012649
1      0.000091  0.005512       8        1   1.692894
3     -0.005250  0.002353       9        1   1.527638
2      0.001053  0.000081      10        1   0.002881, topic_info=        Term        Freq       Total Category  logprob  loglift
199   mother  586.000000  586.000000  Default  30.0000  30.0000
993   libido  390.000000  390.000000  Default  29.0000  29.0000
186      man  261.000000  261.000000  Default  28.0000  28.0000
1389  symbol  204.000000  204.000000  Default  27.0000  27.0000
1525   world  220.000000  220.000000  Default  26.0000  26.0000
...      ...         ...         ...      ...      ...      ...
2216    fire    0.003362  123.274147  Topic10  -5.9628  -0.0546
1299  sexual    0.003338  141.913205  Topic10  -5.9701  -0.2027
376      God    0.003455  156.913041  Topic10  -5.9357  -0.2688
322      way    0.002778   96.400939  Topic10  -6.1535   0.0006
908    horse    0.002933  115.733100  Topic10  -6.0993  -0.1280

[831 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
3744      1  0.398103      =
3744      2  0.243998      =
3744      3  0.089894      =
3744      4  0.102736      =
3744      5  0.064210      =
...     ...       ...    ...
3715      3  0.353454      ἡ
3719      1  0.433639  ἥλιον
3719      3  0.433639  ἥλιον
4988      1  0.337683   Ἰθάς
4988      4  0.337683   Ἰθάς

[2146 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 7, 10, 6, 1, 9, 5, 2, 4, 3])

In [28]:
# mini-conclusion: there aren't many new information, each topic is apparently almost identical, high proportion of 'mother' and 'libido'.

# A bit deeper with 'mother'

In [29]:
# from the previous topic modeling section, the word 'mother' appeared to hold a fair amount of importance, the word probably 
# has some significant connections to different aspects within the text.

# try to examine the contexts in which the word 'mother' appeared
matcher = Matcher(nlp.vocab)
pattern = [
    {'TEXT': {'REGEX': '\w+'}, 'OP': '*'},
    {'TEXT': 'mother', "OP": "+"}, 
    {'TEXT': {'REGEX': '\w+'}, 'OP': '*'}
    ]
matcher.add("matching", [pattern], greedy="LONGEST")
matches = matcher(doc)
matches.sort(key = lambda x: x[1])

for match in matches[:10]:
    print(match, doc[match[1]:match[2]])

(1221037237276548748, 639, 646) that the mother can be the all
(1221037237276548748, 1724, 1733) We know the dream of the mother of Augustus
(1221037237276548748, 6526, 6534) is the mother of the modern scientific attitude
(1221037237276548748, 9278, 9291) as in Grimm’s Fairy Tales which the mother tells to her children
(1221037237276548748, 15419, 15431) It seemed to me as if my mother’s voice wakened me
(1221037237276548748, 18340, 18347) read aloud to me by my mother
(1221037237276548748, 18702, 18706) mother of uncounted peoples
(1221037237276548748, 21613, 21630) it seems to be the influence of the mother transference which creates the attributes of the divinity
(1221037237276548748, 30151, 30172) At the very first and in foremost position it was father and mother who were the objects of the childish love
(1221037237276548748, 30212, 30219) mother imago is organized into a system


In [30]:
# from above 'mother's were used for different meanings, for instance in the context of a mother of a human child, the mother earth, 
# mothers in symbolism, mothers in myths.

# look for names of mothers
pattern = r"mother [A-Z]\w+"
matches = re.finditer(pattern, text)
for match in matches:
    print(match)

<re.Match object; span=(190660, 190670), match='mother Eve'>
<re.Match object; span=(317947, 317959), match='mother Earth'>
<re.Match object; span=(416042, 416053), match='mother Isis'>
<re.Match object; span=(418757, 418768), match='mother Isis'>
<re.Match object; span=(570407, 570421), match='mother Wenonah'>
<re.Match object; span=(713652, 713665), match='mother Tiâmat'>


In [31]:
# mini-conclusion: the multiplicity of interpretations of the word may be one of the factor that contribute to the high frequency of 'mother'